In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install nlp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 35.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 14.2 MB/s eta 0:00:00


In [3]:
!pip install shap
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 572.4/572.4 KB 14.6 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 73.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 82.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 23.6 MB/s eta 0:00:00


In [4]:
import torch
import pandas as pd
import numpy as np
import shap
from transformers import BertTokenizer
import scipy as sp
import transformers
import torch.nn as nn

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
!nvidia-smi

Mon Apr  3 18:36:21 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P8    10W /  70W |      3MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [6]:
model=torch.load(r'/content/drive/MyDrive/Saved Models/MuRIL.pt',map_location=torch.device('cuda'))

In [7]:
tokenizer = BertTokenizer.from_pretrained('google/muril-base-cased', do_lower_case=True)

In [8]:
isinstance(model, nn.DataParallel)

True

In [13]:
def adapter(data):
  #inputs = tokenizer(data, return_tensors="pt")
  inputs = tokenizer(data, 
          return_tensors='pt', 
          padding=True, 
          truncation=True, 
          max_length=128)

  with torch.no_grad():
      logits = model(**inputs).logits
      scores = model(**inputs)[0].softmax(1).detach().cpu().numpy()

  predicted_class_id = logits.argmax().item()
  classifications = ['Introduction','Depression','Grey Area']
  classification = logits.argmax().item()
  class_name = classifications[classification]
  return class_name

In [9]:
def predictor(x):
    probas = model(**tokenizer(x, return_tensors="pt", padding=True))[0].softmax(1).detach().cpu().numpy()
    # logits = model(**tokenizer(x, return_tensors="pt", padding=True)).logits
    # predicted_class_id = logits.argmax().item()
    # classifications = ['Introduction','Depression','Grey Area']
    # classification = logits.argmax().item()
    # class_name = classifications[classification]
    # print(class_name)
    val = sp.special.logit(probas[:,1])
    return val

def f_batch(x):
    val = np.array([])
    for i in x:
      val = np.append(val, predictor(i))
    return val



In [10]:
explainer_bert = shap.Explainer(f_batch, tokenizer)

In [14]:
def explain(data):
  test = { 'text': [data]}
  shap_values = explainer_bert(test)
  print(adapter(data))
  shap.plots.text(shap_values[0])


In [16]:
explain('मैं न्यूयॉर्क में अपने दादा-दादी के साथ पैदा हुई और पली-बढ़ी')

  0%|          | 0/306 [00:00<?, ?it/s]

Introduction


In [18]:
explain('मैं किशोरावस्था से ही गंभीर अवसाद के दौर से गुजर रहा हूं')

Depression


In [19]:
explain('मुझे शौक के तौर पर खाना बनाना पसंद है')

Introduction


In [20]:
explain('मैं इन दिनों बहुत कम महसूस कर रहा हूं, ऐसा महसूस हो रहा है कि मेरा जीवन एक ब्लैक-होल है')

  0%|          | 0/498 [00:00<?, ?it/s]

Partition explainer: 2it [00:14, 14.10s/it]               

Depression


In [21]:
explain('हाल ही में ब्रिटेन की महारानी एलिजाबेथ का निधन हो गया')

Grey Area


In [22]:
explain('मुझे कभी-कभी बहुत दुख होता है लेकिन यह ठीक है')

Depression


In [23]:
explain('पर्याप्त नींद लेने के बाद भी मुझे हर समय थकान महसूस होती है')

Depression


In [24]:
explain('वसंत इतना खुशनुमा मौसम है, सब कुछ एकदम सही है')

  0%|          | 0/182 [00:00<?, ?it/s]

Depression


In [25]:
explain('कविता और कला वह भाषा है जिसे आप बोलते हैं')

  0%|          | 0/132 [00:00<?, ?it/s]

Introduction


In [26]:
explain('ब्यूटी एंड द बीस्ट फिल्म डिज्नी द्वारा बनाई गई थी')

Introduction


In [27]:
explain('कभी-कभी मेरे पास करने के लिए बहुत सारे काम होते हैं जिससे मुझे थकान महसूस होती है, काश मैंने कोई अलग विषय चुना होता')

  0%|          | 0/498 [00:00<?, ?it/s]

Partition explainer: 2it [00:11, 11.42s/it]               

Depression


In [28]:
explain("कभी-कभी कुछ करने का मन नहीं करता, बस एक कोने में बैठ कर रोता हूँ")

  0%|          | 0/420 [00:00<?, ?it/s]

Depression


In [29]:
explain("कभी-कभी आपको केवल अपने दिन के माध्यम से जाने की थोड़ी सी आशा की आवश्यकता होती है")

  0%|          | 0/420 [00:00<?, ?it/s]

Grey Area


In [30]:
explain("जब मैं 16 साल का था तब पता चला कि मुझे ट्यूमर है।")

  0%|          | 0/240 [00:00<?, ?it/s]

Introduction


In [31]:
explain("समुद्र तटों पर सूर्यास्त प्यारा है")

Introduction


In [32]:
explain("क्या आपको बिल्लियाँ या कुत्ते पसंद हैं?")

  0%|          | 0/132 [00:00<?, ?it/s]

Grey Area


In [33]:
explain("मेरे हॉस्टल में एक बहुत प्यारी बिल्ली है")

Grey Area


In [34]:
explain("गुलाब लाल हैं, वोइलेट नीला हैं")

Grey Area


In [35]:
explain("मैं कभी-कभी लोगों के आसपास चिंतित हो जाता हूं")

Depression


In [36]:
explain("मेरा आखिरी मनोचिकित्सक वास्तव में एक दयालु डॉक्टर था जिसने वास्तव में मेरी बात सुनी")

Grey Area


In [37]:
explain("मैं कल्चरल मीट फैशन शो के लिए फैशन डिजाइनर के तौर पर जा रही हूं")

Introduction


In [38]:
explain("मैं कुछ भी महसूस न करने के बजाय दुखी होना पसंद करूँगा")

Depression


In [39]:
explain("केक बनाने के लिए दूध और चीनी की जरूरत होती है")

Grey Area


In [40]:
explain("एक समय था जब मैं वास्तव में आत्मघाती था")

Depression


In [41]:
explain("मुझे नहीं पता कि क्या करना है, मैं बस मरना चाहता हूं और सब कुछ खत्म कर देना चाहता हूं")

  0%|          | 0/498 [00:00<?, ?it/s]

Partition explainer: 2it [00:12, 12.21s/it]               

Depression


In [42]:
explain("मैंने एक बार सुसाइड प्रिवेंशन हेल्पलाइन को कॉल किया क्योंकि मुझे लगा कि मैं खुद को मार लूंगा")

  0%|          | 0/498 [00:00<?, ?it/s]

Partition explainer: 2it [00:12, 12.70s/it]               

Depression


In [43]:
explain("मेरे पास हर समय सुखद भावनाएं हैं")

Depression


In [44]:
explain("मैंने तुम्हारी आँखों में अपना भाग्य देखा")

Grey Area
